In [1]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from dotenv import load_dotenv
from typing import Optional
from pydantic import ValidationError

from pydantic import BaseModel, Field
from typing import Optional, Type

from BS_Schema import make_StatementOfFinancialPosition_model

import BS_Schema
from importlib import reload

BS_Schema = reload(BS_Schema)

In [3]:
## Import Class
SFP = make_StatementOfFinancialPosition_model(2024)
# 4.2) Inspect its name and fields
print(SFP.__name__)           # → "StatementOfFinancialPosition_2024"
print(list(SFP.__fields__.keys()))

StatementOfFinancialPosition_2024
['year', 'cash_and_short_term_investments_unrestricted_and_restricted', 'accumulated_depreciation', 'net_fixed_assets', 'long_term_investments_unrestricted_and_restricted', 'total_assets', 'short_term_debt', 'long_term_debt', 'pension_and_opeb_liability', 'total_liabilities', 'net_assets_without_donor_restrictions', 'expendable_net_assets_with_donor_restrictions', 'perpetual_net_assets_with_donor_restrictions', 'net_assets_with_donor_restrictions', 'total_net_assets', 'total_liabilities_and_net_assets']


In [5]:
# ee231c03c0cf6607315be41f48f2ca75503002282bb13f37ff28a6197360edcd

# llx-zKFsBQYVXG6DqlTVV2yi3xpkS8nePwYvgOeYoMzJ04RBpgYw

#llx-85vV2s7NGkSWQevqJR6pn6I7hMgksTzbd9I4Vk6qIAPwtmUs - organization key

In [7]:
PDF_ROOT = "scrapping/university_pdfs/"
OUTPUT_ROOT = "output_Balance_Sheet_DS"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "bcb15a18-67ac-4772-9965-2654ecaff88c" #Different based on your LLamaCloud account
load_dotenv() #make sure the API key is in the .env file
Year = 2024
extractor = LlamaExtract(api_key='llx-jlyOr0ZQwOzn0BPYgzggmo6mFShTLTsxdrXGviOaBf6IDnHG',project_id = '8c10e62e-3810-4193-915d-d2d11105826d')# LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

# #uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "balance-sheet-parser-v1", data_schema=SFP)

agent = extractor.get_agent(id = AGENT_ID)

# uncomment the following lines if you updated the schema
agent.data_schema = SFP
agent.save()
agent = extractor.get_agent(id = AGENT_ID)

In [8]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'year': {'description': "Only extract from the full Statement of Financial Position (Balance Sheet); do not use condensed or net financial position tables, and do not perform any calculations—capture the number exactly as shown. The fiscal year to which all other line‐items refer. E.g. '2024' means every balance (Cash, Liabilities, Net Assets, etc.) is 'as of June 30, 2024'.",
   'type': 'integer'},
  'cash_and_short_term_investments_unrestricted_and_restricted': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Only extract from the full Statement of Financial Position (Balance Sheet); do not use condensed or net financial position tables, and do not perform any calculations—capture the number exactly as shown. Total of Cash & Short-Term Investments (both unrestricted and restricted) as of June 30, 2024. May be labeled 'Cash & Short-Term Investments (Unrestricted & Restricted)', 'Cash and investments', or similar.

In [11]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [13]:
# os.listdir('/Users/danyalsoomro/Desktop/Python_Projects/Industry_projects/Franklin_Templton_Industry_Project/scrapping/university_pdfs')

In [17]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT))[0:4]:
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

Processing school: ARIZONA_STATE_UNIVERSITY
Extracting data from FY_2024_Arizona_State_University_Annual_Comprehensive_Financial_Report_for_the_year_ended_06_30_2024__4.1_MB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:23<00:00, 23.95s/it]


Extracting data from FY_2024_Arizona_State_University_Continuing_Disclosure_Undertaking_for_the_year_ended_06_30_2024__160_KB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:07<00:00,  7.81s/it]


Saved output to output_Balance_Sheet_DS/ARIZONA_STATE_UNIVERSITY.xlsx
Processing school: BRADLEY_UNIVERSITY
Extracting data from Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf


...

Extracting data from Exhibit_1_DRAFT_UNAUDITED_CSU_Combined_Financial_Statements_6-30-24_for_the_year_ended_06_30_2024__1.2_MB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:10<00:00, 10.13s/it]


Extracting data from Exhibit_1_DRAFT_UNAUDITED_CSU_Combined_Financial_Statements_6-30-24_for_the_year_ended_06_30_2024__298_KB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:08<00:00,  8.71s/it]

Saved output to output_Balance_Sheet_DS/CALIFORNIA_STATE_UNIVERSITY.xlsx
Extraction complete.


In [ ]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")